In [1]:
import json  
from openai import OpenAI
import base64    
import os
from tqdm import tqdm
import time

In [ ]:

client = OpenAI(
    api_key="sk-mstowpepwkvabxpdxenmcyzdkzvnzolhakhcxkcjsgdkegao", # 从https://cloud.siliconflow.cn/account/ak获取
    base_url="https://api.siliconflow.cn/v1"
)

In [ ]:
# 读取test set
test_file_path = '/data3/maruolong/Train_Data/main/Data/Train_and_Test_Data/CoT_and_Original_Data/20_Paths/gc/CoT_Data/gc_CoT_test.json'
with open(test_file_path) as f:
    data = json.load(f)

data[0]  #一条数据样例

{'sample_id': '601338000006',
 'prompt': "<image>\nYou are provided with a street view image from a U.S. census tract. Based on the architectural style, building materials, cleanliness of the streets, and any other relevant visual indicators (such as vehicle types, landscaping quality, or visible infrastructure), evaluate the income level of the area's residents. Assign a score between 0 and 100, where 100 represents the highest income level. Additionally, estimate the predominant race of the area from the following options: \n- White alone, not Hispanic or Latino \n- Asian \n- Black or African American \n- Hispanic or Latino \n\nProvide your response in **exactly this format**: [income score: estimated race] \nInclude no other information or explanation. Output must strictly follow this format.\nFor example: `85: White alone, not Hispanic or Latino`.",
 'image': ['/data2/ouyangtianjian/NEW_StreetView_Images_US_CUT_merged/US_StreetView_0_to_2500_CUT/6013380000/ZDZB5BWRF2xelfl9ccaSCw&37

In [5]:
# 生成记录结果的文件
answers_file = "/data3/maruolong/Train_Data/Add_Path/Qwen2-VL-7B/train.jsonl"
os.makedirs(os.path.dirname(answers_file), exist_ok=True)
ans_file = open(answers_file, "w")

In [ ]:
# 构建给大模型的input
idx = 0
tract_id = data[idx]['sample_id']
text = data[idx]['prompt']
image_list = data[idx]['image']

# 具体传给大模型的message
messages=[
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": text
            }
        ]
    }]
# 图片用base64库处理后传入
for image in image_list:
        with open(image, "rb") as image_file:
            img_url = "data:image/jpeg;base64," + base64.b64encode(image_file.read()).decode('utf-8')
        messages[0]['content'].append({
            "type": "image_url",
            "image_url": {
                "url": img_url,
                "detail":"low"
            }
        })

In [7]:
# 调用大模型API！
response = client.chat.completions.create(
            model="Qwen/Qwen2-VL-72B-Instruct",
            messages=messages,
    )

In [8]:
description =  response.choices[0].message.content
description=description.strip()
description

'50: White alone, not Hispanic or Latino'

In [9]:
# 记录本次结果
ans_file.write(json.dumps({
                                "tract": tract_id[:-2],
                                "num": int(tract_id[-2:]),
                                "predict": description,                                   
                                },ensure_ascii=False) + "\n")
ans_file.flush()


In [ ]:

answers_file = "/data3/maruolong/Train_Data/main/Data/Train_and_Test_Data/CoT_and_Original_Data/20_Paths/gc/CoT_Data/gc_CoT_test.json"
os.makedirs(os.path.dirname(answers_file), exist_ok=True)
ans_file = open(answers_file, "w")

for idx in tqdm(range(len(data))):
    tract_id = data[idx]['sample_id']
    text = data[idx]['conversations'][0]['value']
    ground_truth = data[idx]['conversations'][1]['value']
    image_list = data[idx]['image']

    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": text
                }
            ]
        }]
    
    for image in image_list:
        try:
            with open(image, "rb") as image_file:
                base64_content = base64.b64encode(image_file.read()).decode('utf-8')
                if not base64_content:  # 检查base64内容是否为空
                    print(f"Empty base64 content for image: {image}. Skipping this image.")
                    continue
                img_url = "data:image/jpeg;base64," + base64_content
            messages[0]['content'].append({
                "type": "image_url",
                "image_url": {
                    "url": img_url,
                    "detail": "low"
                }
            })
        except Exception as e:
            print(f"Error processing image {image}: {e}. Skipping this image.")
            continue

    if not messages[0]['content']:
        print(f"No valid content for sample {tract_id}. Skipping this sample.")
        continue
    try:
         response = client.chat.completions.create(
            model="Qwen/Qwen2-VL-72B-Instruct",
            messages=messages,
         )
    except Exception as e:
         print("Rate limit reached. Retrying in 5 seconds…")
         time.sleep(5)
         response = client.chat.completions.create(
            model="Qwen/Qwen2-VL-72B-Instruct",
            messages=messages,
         ) 

    description =  response.choices[0].message.content
    description=description.strip()

    ans_file.write(json.dumps({
                                "tract": tract_id,
                                "label": ground_truth,
                                "predict": description,                                   
                                },ensure_ascii=False) + "\n")
    ans_file.flush()

 78%|███████▊  | 25046/32052 [10:21:40<2:45:35,  1.42s/it] 